# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read in clean_weather_data.csv
weather_data_filepath = "resource/clean_weather_data.csv"
weather_data = pd.read_csv(weather_data_filepath, index_col = 0)
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,atuona,-9.8000,-139.0333,298.90,75,0,7.32,PF,1644522253
1,puerto ayora,-0.7393,-90.3518,302.13,85,96,3.13,EC,1644522254
2,punta arenas,-53.1500,-70.9167,285.16,57,40,12.86,CL,1644522254
3,birigui,-21.2886,-50.3400,302.20,54,72,3.13,BR,1644522121
4,rikitea,-23.1203,-134.9692,299.38,72,89,8.41,PF,1644522255
...,...,...,...,...,...,...,...,...,...
551,paamiut,61.9940,-49.6678,262.71,73,100,3.97,GL,1644523232
552,lagoa,39.0500,-27.9833,287.91,73,100,2.46,PT,1644523233
553,isangel,-19.5500,169.2667,299.76,90,100,10.25,VU,1644523233
554,dzaoudzi,-12.7887,45.2699,298.09,94,100,1.54,YT,1644523234


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

locations = weather_data[["Lat", "Lng"]]
weight = weather_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
